In [1]:
# import warnings
# warnings.filterwarnings("ignore") 

In [2]:
# from dask_gateway import Gateway
# from dask.distributed import Client
# gateway = Gateway()
# cluster = gateway.new_cluster()
# cluster.adapt()
# client = Client(cluster)
# cluster


# Find coef

In [2]:
import numpy as np
import xarray as xr
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
#load datasets as xarrays
roy_in = xr.open_dataset('/home/jovyan/roy and tk data/roy_data.nc')
tk_in = xr.open_dataset('/home/jovyan/roy and tk data/tk_data.nc')

In [4]:
months = ["Jan","Feb","Mar","Apr","May","Jun","Jul",
          "Aug","Sep","Oct","Nov","Dec"]

In [5]:
print(4320/360) # 12 points in each degree longitude
print(2160/180) # 12 points in each degree latitude

12.0
12.0


In [6]:
# onanmeanongitude, a for latitude
# l for lowbound h for highbound
o_l = 100 # longitude low bound 
o_h = 4300 # lon high
a_l = 100 # lat low
a_h = 2100 # lat high


In [7]:
# roy_in.sel(year=0).to_array()


In [8]:
# convert both xarray datasets to 3d numpy arrays
years = len(roy_in.year)

for i in range(years): # for each year
    
    # create temporary list of arrays for each year
    # 12 arrays in list for each year
    # slices of global determined earlier are array bounds for each array
    # arrays are numpy so months found through index not var identifier
    roy_temp = roy_in.sel(year = i).to_array()[:,o_l:o_h,a_l:a_h]
    tk_temp = tk_in.sel(year = i).to_array()[:,o_l:o_h,a_l:a_h]
    print('year = ', i+1) # for checking progress

    for j in range(len(months)): # each month within each year
        
                
        if i == 0 and j == 0: # year 1 month 1: initiating array
            roy_array = roy_temp[j] # one array for each dataset
            tk_array = tk_temp[j]
        
        elif i == 0 and j == 1: # year 1 month 2: adding 2d array to 3d
            roy_array = np.stack((roy_array,roy_temp[j])) # stack 2 2d 
            tk_array = np.stack((tk_array,tk_temp[j])) # creates 3d arrays
        
        else: # add further 2d arrays to 3d array

            roy_array = np.concatenate((roy_array,[roy_temp[j]]))
            tk_array = np.concatenate((tk_array,[tk_temp[j]]))
            

year =  1
year =  2
year =  3
year =  4
year =  5


In [9]:
# remove outliers from both arrays
# done on basis of each time step (60 total) rather than entire array

print('point___roy_________________tk______________') # for showing maxs

for i in range(len(roy_array)): # through each timestep
    
    max_roy = np.nanpercentile(roy_array[i],99) # find limit of 99th percentile
    max_tk = np.nanpercentile(tk_array[i],99) # done on both arrays
    
    print(i,'    ',max_roy,' ',max_tk) # for showing values
    
    # in each array, in the given z value (timestep)
    # for values in array less than max, preserve original value
    # otherwise replace with nan value
    roy_array[i] = np.where(roy_array[i]<max_roy,roy_array[i],np.nan)
    tk_array[i] = np.where(tk_array[i]<max_tk,tk_array[i],np.nan)

point___roy_________________tk______________
0      62.444264444469596   223.0915653118241
1      61.026244538904116   186.51807964035484
2      68.19501724384746   177.91775686606374
3      87.71085559841585   192.62805743126106
4      124.84093032963813   211.68413497847737
5      98.92517932821211   246.3580401197868
6      125.12192460203273   227.95838747944484
7      139.31264744010087   244.83402205183484
8      101.89966119702062   215.3527976502453
9      79.55075377305181   195.74980114146436
10      64.3459923076707   186.35702408268367
11      67.19546684064446   185.19846714458998
12      61.04649344446801   222.6415142892931
13      65.40708031098654   224.51498115229253
14      64.17885248980478   197.23290070630796
15      85.13885353195587   206.0085605769529
16      130.56092456318868   242.97231505848313
17      107.82664807173916   264.5114382843216
18      113.90273087091302   252.76936745480882
19      132.14368155900104   270.833891699752
20      111.315306870676

In [9]:
# roy_in.Dec.sel(year=1).values[500:514,500:515]

In [10]:
#check no negative values
#success: no negatives!

# for i in range(len(roy_in.year)):
#     display(np.amin(roy_in.sel(year=i)))
#     display(np.amin(tk_in.sel(year=i)))

In [21]:
#below doesnt work for some reason (only on tk)
#xr.where(tk_in<max_tk,tk_in,np.nan)

# For correlation of entire map
ignore until next title

In [ ]:
#flatten 3d arrays to 1d for correlation
roy_flat = np.ndarray.flatten(roy_array[:])
tk_flat = np.ndarray.flatten(tk_array[:])

In [ ]:
# create a dataframe (starts 2 by pixel number times 60)
combine = [roy_flat,tk_flat]
df = pd.DataFrame(combine,index=['roy','tk'])

In [14]:
# must transpose the matrix to be pixels by datasets for correlation
# calculate correlation matrix
df = df.transpose()
corr_mat = df.corr()

In [15]:
# parse of coefficient
coef = corr_mat.values[0,1]

## Working with coefficient

In [16]:
k_roy = coef * roy_in


In [17]:
# display(roy_in.Jan.sel(year=0)[500:505,500:505])
# display(coef * 16.15)
# k_roy.Jan.sel(year=0)[500:505,500:505]

In [18]:
k_array = coef * roy_array

In [23]:
difference = tk_array - k_array

# correlations for each pixel

In [10]:
lon_size = roy_array.shape[1]
lat_size = roy_array.shape[2]

print(lon_size)
print(lat_size)

corr_array = np.zeros((lon_size,lat_size))


4200
2000


In [ ]:
#in our 3d array we want to create a correlation coefficient for each (x,y) pixel value
#we can use df.corr along z axis for each (x,y) value

for i in range(lon_size):

    if np.mod(i,50) == 0:
        print((i/lon_size)*100,'%')   
        
    for j in range(lat_size):
        
        roy_z = roy_array[:,i,j] #absorption
        tk_z = tk_array[:,i,j] #backscattering
        
        z_combined = [roy_z,tk_z]
        corr_df = pd.DataFrame(z_combined,index = ['roy','tk'])
        corr_df = corr_df.transpose()
        
        corr_array[i,j] = corr_df.corr().values[0,1]  
        


0.0 %
1.1904761904761905 %
2.380952380952381 %
3.571428571428571 %
4.761904761904762 %
5.952380952380952 %
7.142857142857142 %
8.333333333333332 %


In [ ]:
# to remove seasonality average each month over years (12 step time series)
# look into regressoin coefficients

In [ ]:
corr_array = np.transpose(corr_array)

In [ ]:
plt.imshow(corr_array)
plt.colorbar()
plt.savefig('/home/jovyan/quickplots/roy_tk_['
            +str(o_l)+'-'+str(o_h)+','
            +str(a_l)+'-'+str(a_h)+
            '].png', dpi=600,format='png')

In [ ]:
np.nanmean(corr_array)

# Remove seasonality influence

In [ ]:
lon_size = roy_array.shape[1]
lat_size = roy_array.shape[2]

print(lon_size)
print(lat_size)

corr_array = np.zeros((lon_size,lat_size))


In [ ]:
roy_season = np.zeros((len(months),lon_size, lat_size))
tk_season = np.zeros((len(months),lon_size, lat_size))

In [ ]:
groups = np.zeros((5,12))

for i in range(years):
    groups[i] = np.arange((12*i),(12*(i+1)))

    
groups = np.transpose(groups)
groups = np.array(groups, dtype = 'int')

In [ ]:
roy_array.shape

In [ ]:
# roy_grouping = roy_array[groups,i,j]

# display(roy_array[groups,i,j])
# display(roy_array[:,i,j])
# display(np.nanmean(roy_grouping,axis=1))

In [ ]:
for i in range(lon_size):
    
    for j in range(lat_size):
        
        roy_grouping = roy_array[groups,i,j]
        tk_grouping = tk_array[groups,i,j]
        
        roy_season[:,i,j] = np.nanmean(roy_grouping,axis=1)
        tk_season[:,i,j] = np.nanmean(tk_grouping,axis=1)


In [ ]:
roy_season.shape

In [ ]:
seas_lon = roy_season.shape[1]
seas_lat = roy_season.shape[2]

print(seas_lon)
print(seas_lat)

season_corr = np.zeros((seas_lon,seas_lat))


In [ ]:
#in our 3d array we want to create a correlation coefficient for each (x,y) pixel value
#we can use df.corr along z axis for each (x,y) value

for i in range(seas_lon):

    if np.mod(i,50) == 0:
        print((i/seas_lon)*100,'%')   
        
    for j in range(seas_lat):
        
        r_z_seas = roy_season[:,i,j] #absorption
        t_z_seas = tk_season[:,i,j] #backscattering
        
        z_seas_combined = [r_z_seas,t_z_seas]
        corr_df_seas = pd.DataFrame(z_seas_combined,index = ['roy','tk'])
        corr_df_seas = corr_df_seas.transpose()
        
        season_corr[i,j] = corr_df_seas.corr().values[0,1]  
        


In [ ]:
season_corr = np.transpose(season_corr)

In [ ]:
plt.imshow(season_corr)
plt.colorbar()
plt.savefig('/home/jovyan/quickplots/'+
            'detrended_roy_tk_['
            +str(o_l)+'-'+str(o_h)+','
            +str(a_l)+'-'+str(a_h)+
            '].png', dpi=600,format='png')

In [ ]:
np.nanmean(season_corr)